In [1]:
import os
import shutil
import pandas as pd
import biogeme.biogeme as bio
from biogeme.models import loglogit
import pycmtensor as cmt
from pycmtensor.expressions import Beta

nb_path = os.path.abspath("")
model_average_sp = pd.read_csv("data/model_average_SP.csv")
model_average_sp.columns = model_average_sp.columns.str.upper()
model_average_sp[["AV1", "AV2"]] = 1
db = cmt.Database("model_average_sp", model_average_sp, choiceVar="CHOICE")
globals().update(db.variables)

# additional steps to format database
db.data['CHOICE'] -= 1
db.choices = sorted(db.data["CHOICE"].unique()) # save original choices
db.autoscale(list(db.data.loc[:, 'TT1': 'HH_INC_ABS'].columns), verbose=False)

b_cost = Beta("b_cost", 0.0, None, None, 0)
b_time = Beta("b_time", 0.0, None, None, 0)
b_hw = Beta("b_hw", 0.0, None, None, 0)
b_ch = Beta("b_ch", 0.0, None, None, 0)
b_hh_inc1 = Beta("b_hh_inc1", 0.0, None, None, 0)
b_hh_inc2 = Beta("b_hh_inc2", 0.0, None, None, 1)
b_carav1 = Beta("b_carav1", 0.0, None, None, 0)
b_carav2 = Beta("b_carav2", 0.0, None, None, 1)
b_com = Beta("b_com", 0.0, None, None, 0)
b_shop = Beta("b_shop", 0.0, None, None, 0)
b_bis = Beta("b_bis", 0.0, None, None, 0)
b_lei = Beta("b_lei", 0.0, None, None, 0)
asc_alt1 = Beta("asc_alt1", 0.0, None, None, 1)
asc_alt2 = Beta("asc_alt2", 0.0, None, None, 0)

U_1 = (
    b_cost * TC1 + b_time * TT1 + b_hw * HW1 + b_ch * CH1 
    + b_hh_inc1 * HH_INC_ABS + b_carav1 * CAR_AVAILABILITY
    + b_com * COMMUTE + b_shop * SHOPPING + b_bis * BUSINESS + b_lei * LEISURE
    + asc_alt1
)
U_2 = (
    b_cost * TC2 + b_time * TT2 + b_hw * HW2 + b_ch * CH2 
    + b_hh_inc2 * HH_INC_ABS + b_carav2 * CAR_AVAILABILITY
    + asc_alt2
)

# Associate utility functions with the numbering of alternatives
V = {0: U_1, 1: U_2}

# Associate the availability conditions with the alternatives
av = {0: AV1, 1: AV2}

# Definition of the model. This is the contribution of each
# observation to the log likelihood function.
logprob = loglogit(V, av, CHOICE)

# Create the Biogeme object
biogeme = bio.BIOGEME(db, logprob)
biogeme.modelName = 'logit_SP'

# Calculate the null log likelihood for reporting.
biogeme.calculateNullLoglikelihood(av)

if os.path.isdir(biogeme.modelName):
    shutil.rmtree(biogeme.modelName)
try:
    os.mkdir(biogeme.modelName)
    os.chdir(biogeme.modelName)
    
    # Estimate the parameters
    results = biogeme.estimate()
    os.chdir("..")
except:
  print("An exception occurred")

# Get the results in a pandas table
pandasResults = results.getEstimatedParameters()

print(results)
print(pandasResults)


Results for model logit_SP
Output file (HTML):			logit_SP.html
Nbr of parameters:		11
Sample size:			2790
Excluded data:			0
Null log likelihood:		-1933.881
Init log likelihood:		-1933.881
Final log likelihood:		-1324.212
Likelihood ratio test (null):		1219.337
Rho square (null):			0.315
Rho bar square (null):			0.31
Likelihood ratio test (init):		1219.337
Rho square (init):			0.315
Rho bar square (init):			0.31
Akaike Information Criterion:	2670.425
Bayesian Information Criterion:	2735.697
Final gradient norm:		0.007185987
asc_alt2       : -0.0887[0.0914 -0.971 0.332][0.093 -0.953 0.34]
b_bis          : -0.114[0.145 -0.783 0.434][0.147 -0.776 0.438]
b_carav1       : -0.281[0.103 -2.72 0.00656][0.102 -2.75 0.00604]
b_ch           : -1.15[0.0486 -23.6 0][0.0507 -22.6 0]
b_com          : 0.121[0.0898 1.35 0.177][0.0871 1.39 0.163]
b_cost         : -11.6[1.49 -7.79 6.66e-15][2.06 -5.63 1.76e-08]
b_hh_inc1      : -0.0602[0.113 -0.534 0.594][0.115 -0.524 0.6]
b_hw           : -0.386[0.0209